In [24]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
import math

In [25]:
# Importando .csv
df = pd.read_csv('dataset/train_data.csv')
movies = pd.read_csv('dataset/movies_data.csv')
users = pd.read_csv('dataset/users_data.csv')

# Criando Matrix User X Movie
df_u_m = df.pivot(index='user_id', columns='movie_id', values='rating')
df_u_m = pd.DataFrame(df_u_m, index=users["user_id"], columns=movies["movie_id"])

In [26]:
# No SVD, o 0 representa valores desconhecidos
df_u_m.fillna(value=0, inplace=True)
U, s, Vh = np.linalg.svd(df_u_m.values)

In [27]:
def predict(u,i, k = 5):
    p2 = U[u, :k]
    q2 = Vh[i, :k]
    s2 = s[:k]
    # r = 0
    # for i in range(len(u)):
    #     r += u[i]*s[i]*v[i]
    # return(r)
    return p2.dot(np.diag(s2)).dot(q2)

In [28]:
# Avaliando predições
def rmse(test):
    sum_err = 0
    for t in test:
        u = t[0]-1
        i = t[1]-1
        r_ui = t[2]
        pred = predict(u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [29]:
rmse(df.values[:10])

4.015704107152739